# Neural Networkin Tensorflow

In [1]:
import tensorflow as tf;
import sklearn.datasets;
iris_ds = sklearn.datasets.load_iris(return_X_y=False)
import pandas as pd

In [2]:
iris_ds.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [3]:
import numpy as np
iris_data = pd.DataFrame(data=iris_ds.data,columns=iris_ds.feature_names)

In [4]:
iris_data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
import sklearn;
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
scaled_data = min_max_scaler.fit_transform(iris_data)


In [7]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(3)
label = encoder.fit_transform(iris_ds.target.reshape(-1,1))
label = label.todense()

C:\Users\achintj\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)


In [8]:

trainx,testx,trainy,testy = train_test_split(scaled_data,label)

In [9]:
print (testx.shape)

(38, 4)


#### Building a model in Tensorflow

In [10]:
import tensorflow as tf;
x = tf.placeholder(shape=[None,4],dtype=tf.float32)
y = tf.placeholder(shape=[None,3],dtype=tf.float32)
w0 = tf.get_variable('w0',shape=[4,5],dtype=tf.float32)
w1 = tf.get_variable('w1',shape=[5,3],dtype=tf.float32)
b0 = tf.get_variable('b0',shape=[1,5],dtype=tf.float32)
b1 = tf.get_variable('b1',shape=[1,3],dtype=tf.float32)

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
def build_model(x):
    with tf.name_scope('hidden'):
        h = tf.nn.relu(tf.matmul(x,w0)+b0)
        print (h.shape)
    with tf.name_scope('output'):
        yhat = tf.nn.softmax(tf.matmul(h,w1)+b1)
        print (yhat.shape)
    return yhat

In [12]:
# Applying build_model

y_hat = build_model(x)

(?, 5)
(?, 3)


In [13]:
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y,1),tf.argmax(y_hat,1)),tf.float64))
    tf.summary.scalar('accuracy',accuracy)

In [14]:
with tf.name_scope('entropy-loss'):
    entropy_loss = tf.losses.softmax_cross_entropy(logits=y_hat,onehot_labels=y)
    tf.summary.scalar('entropy-loss',entropy_loss)


Instructions for updating:
Use tf.cast instead.


In [15]:
writer = tf.summary.FileWriter('nn',tf.get_default_graph())

In [16]:
for lr in [0.1,0.05,0.01,0.005,0.001]:
    writer1 = tf.summary.FileWriter('nn/lr-'+str(lr))
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(entropy_loss)
        merged = tf.summary.merge_all()
    with tf.Session() as sess :
        sess.run(tf.global_variables_initializer())
        #sess.run(tf.local_variables_initializer())
        for j in range(300):
            for i in range(len(trainx)):
                x_val = np.array(trainx[i].reshape(1,-1),dtype=np.float32)
                y_val = np.array(trainy[i].reshape(1,-1),dtype=np.float32)
                #print (i,x_val,y_val)
                #print (x_val.dtype,'\n',y_val.dtype)
                optimizer.run(feed_dict={x:x_val,y:y_val})
                _loss, = sess.run([entropy_loss] ,feed_dict={x:x_val,y:y_val})
                #print ('yhat,y',_y_hat,y_val)
                #print ('accuracy',_acc)
            if(j%5==0):
                summary,_train_acc = sess.run([merged,accuracy],feed_dict={x:trainx,y:trainy})
                writer1.add_summary(summary,j)
                _test_acc = sess.run(accuracy,feed_dict={x:testx,y:testy})
                print ('loss=%2.6f, train accuracy=%2.6f, test accuracy = %2.6f'%(_loss,_train_acc,_test_acc))

loss=0.951758, train accuracy=0.785714, test accuracy = 0.789474
loss=0.551669, train accuracy=0.892857, test accuracy = 0.973684
loss=0.551450, train accuracy=0.955357, test accuracy = 1.000000
loss=0.551445, train accuracy=0.955357, test accuracy = 1.000000
loss=0.551456, train accuracy=0.973214, test accuracy = 1.000000
loss=0.551445, train accuracy=0.946429, test accuracy = 1.000000
loss=0.551445, train accuracy=0.973214, test accuracy = 1.000000
loss=0.551445, train accuracy=0.955357, test accuracy = 1.000000
loss=0.551445, train accuracy=0.955357, test accuracy = 1.000000
loss=0.551445, train accuracy=0.982143, test accuracy = 1.000000
loss=0.551445, train accuracy=0.955357, test accuracy = 0.973684
loss=0.551445, train accuracy=0.946429, test accuracy = 1.000000
loss=0.551445, train accuracy=0.982143, test accuracy = 1.000000
loss=0.551445, train accuracy=0.946429, test accuracy = 0.947368
loss=0.551445, train accuracy=0.964286, test accuracy = 1.000000
loss=0.551445, train accu

loss=0.551741, train accuracy=0.982143, test accuracy = 1.000000
loss=0.551669, train accuracy=0.982143, test accuracy = 1.000000
loss=0.551622, train accuracy=0.982143, test accuracy = 1.000000
loss=0.551589, train accuracy=0.982143, test accuracy = 1.000000
loss=0.551565, train accuracy=0.982143, test accuracy = 1.000000
loss=0.551548, train accuracy=0.982143, test accuracy = 1.000000
loss=0.551521, train accuracy=0.982143, test accuracy = 1.000000
loss=0.551501, train accuracy=0.982143, test accuracy = 1.000000
loss=0.551487, train accuracy=0.973214, test accuracy = 1.000000
loss=0.551476, train accuracy=0.973214, test accuracy = 1.000000
loss=0.551470, train accuracy=0.973214, test accuracy = 1.000000
loss=0.551465, train accuracy=0.973214, test accuracy = 1.000000
loss=0.551461, train accuracy=0.973214, test accuracy = 1.000000
loss=0.551458, train accuracy=0.973214, test accuracy = 1.000000
loss=0.551456, train accuracy=0.973214, test accuracy = 1.000000
loss=0.551455, train accu

KeyboardInterrupt: 

In [ ]:
with tf.name_scope('myscope'):
    a = tf.constant(name='const1',value=3)
print (a.name)

In [ ]:
1x4096

conv1d(1x4096)(1x2)(1x4096x40) relu
pool2(1x2043x40)
conv1d(1x2096x40)(1x2)(1x2096x80) relu
pool2(1x1022x80)

dense(1x2000) relu
dense(1x52) softmax (52)
